## p-value

In [96]:
# 예시
a = [0, 0, 0, 1, 1, 1, 1]
b = [0, 1, 2, 3, 4, 5, 6]
stats.pearsonr(a, b)

(0.8660254037844386, 0.011724811003954654)

앞이 상관계수 뒤가 p-value

In [97]:
res = pd.DataFrame(columns = [0,1,2,3])
for i in range(cnt.shape[1]-4, cnt.shape[1]):
    for j in range(cnt.shape[1]-4):
        pearr = stats.pearsonr(cnt.iloc[:,i:i+1].iloc[:,0],cnt.iloc[:,j:j+1].iloc[:,0])
        if(pearr[1] < 0.05):
            temp = pd.DataFrame([cnt.columns[i], cnt.columns[j], pearr[0], pearr[1]]).T
            res = res.append(temp)            

In [98]:
# 가중치 준 버전 - count
cnt = cnt_w.copy()
res = pd.DataFrame(columns = [0,1,2,3])
for i in range(cnt.shape[1]-4, cnt.shape[1]):
    for j in range(cnt.shape[1]-4):
        pearr = stats.pearsonr(cnt.iloc[:,i:i+1].iloc[:,0],cnt.iloc[:,j:j+1].iloc[:,0])
        if(pearr[1] < 0.05):
            temp = pd.DataFrame([cnt.columns[i], cnt.columns[j], pearr[0], pearr[1]]).T
            res = res.append(temp)            

In [99]:
# 가중치 준 버전 - mean
cnt = mean_w.copy()
res = pd.DataFrame(columns = [0,1,2,3])
for i in range(cnt.shape[1]-4, cnt.shape[1]):
    for j in range(cnt.shape[1]-4):
        pearr = stats.pearsonr(cnt.iloc[:,i:i+1].iloc[:,0],cnt.iloc[:,j:j+1].iloc[:,0])
        if(pearr[1] < 0.05):
            temp = pd.DataFrame([cnt.columns[i], cnt.columns[j], pearr[0], pearr[1]]).T
            res = res.append(temp)            

In [102]:
print(res.shape)
res.head()

(474, 4)


,0,1,2,3
0,oil_GS,mean_11번가,0.438953,1.11902e-33
0,oil_GS,mean_BAND,0.340176,4.80358e-20
0,oil_GS,mean_CBS 레인보우,0.119933,0.00165022
0,oil_GS,mean_CJ온마트,0.503338,2.43699e-45
0,oil_GS,mean_Cyworld,0.0836726,0.0284253


In [103]:
res.columns = ["Source","Target","Weight","p-value"]
res["Weight"] = abs(res["Weight"])

## Gephi 그리기

- 1. 어플끼리만
- 2. 어플 카테고리로 묶어서
- 3. 결제 카테고리로 묶어서 (store 빼고 cat만)
- 4. 결제 store끼리

In [53]:
# 기본 데이터
app = cnt[cnt.columns[cnt.columns.str.contains("cnt")]]
pay = cnt[cnt.columns[[x[:3] == "PAY" for x in cnt.columns]]]
target = cnt.iloc[:,-4:]
# 패널/gender/age_10/oil 데이터
meta_data = oil_r[["PANEL_ID", "gender","age_10","CATEGORY_GROUP_CODE_231","CATEGORY_GROUP_CODE_232","CATEGORY_GROUP_CODE_233","CATEGORY_GROUP_CODE_234"]]
meta_data.columns = ["panel_id", "gender","age_10","oil_GS","oil_SK","oil_S-oil","oil_HD"]
# 패널목록
panel_list = list(meta_data["panel_id"])

#### 0. 어플 이름으로

In [90]:
app0 = pd.read_csv("data/PANEL/APP Refine_Usage_Sum_Time.csv", encoding = "cp949")

In [156]:
# oil 패널들만 남기기
app0["index"] = app0.apply(lambda x: 1 if x["panel_id"] in panel_list else 0 , axis = 1)
app0_oil = app0[app0["index"]==1]
# n : 초기 필터링 개수
n = 200
app_usage = app0_oil.groupby("app_name").count().reset_index()
app_usage = app_usage[app_usage["panel_id"] > n]
# 사용할 어플 : 어플사용수 dictionary 화
app_dict = app_usage[["app_name","reg_cnt"]].set_index("app_name").to_dict()["reg_cnt"]
app_list = list(app_dict.keys())
# 사용할 어플만 남기기
app0_oil["index"] = app0_oil.apply(lambda x : 1 if x["app_name"] in app_list else 0, axis = 1)
app0_oil = app0_oil[app0_oil["index"] == 1]
# pivot table
app0_pivot = pd.pivot_table(app0_oil, index = "panel_id", columns = "app_name", values= "total_time_stamp", aggfunc = np.mean).reset_index().fillna(0)
# meta data 붙이기
meta_data.columns = ['panel_id', 'gender', 'age_10', 'oil_GS', 'oil_SK', 'oil_S-oil','oil_HD']
app0_pivot = pd.merge(app0_pivot, meta_data, on ="panel_id")
# 가중치 곱하기
app0_pivot = app0_pivot.drop("panel_id", axis = 1)
app0_w = app0_pivot.apply(lambda x: x * (weight.loc[str(int(x["gender"]))+"-"+str(int(x["age_10"]))]["weight"]), axis = 1)
app0_w = app0_w.drop(["age_10","gender"], axis =1)

In [157]:
# gephi 모양 만들기
cnt = app0_w.copy()

res = pd.DataFrame(columns = [0,1,2,3])
for i in range(cnt.shape[1]-4, cnt.shape[1]):
    for j in range(cnt.shape[1]-4):
        pearr = stats.pearsonr(cnt.iloc[:,i:i+1].iloc[:,0],cnt.iloc[:,j:j+1].iloc[:,0])
        if(pearr[1] < 0.05):
            temp = pd.DataFrame([cnt.columns[i], cnt.columns[j], pearr[0], pearr[1]]).T
            res = res.append(temp)            
res.columns = ["Source","Target","Weight","p-value"]
# size 넣기
oil_list = ["oil_GS","oil_SK","oil_S-oil","oil_HD"]
res_node = pd.DataFrame()
res_node["node"] = list(res["Source"].unique()) + list(res["Target"].unique())
res_node["size"] = res_node.apply(lambda x : 50 if x["node"] in oil_list else 20, axis = 1)
# 하나의 주유소와만 연결된 애들 찾기
temp = res.groupby("Target").count().reset_index()
once_list = list(temp[temp["Source"] == 1]["Target"])
res_node["color"] = res_node.apply(lambda x: 0 if x["node"] in once_list else x["size"], axis = 1)
res_node.columns = ["Id", "size","color"]
res_node["Label"] = res_node["Id"]

In [223]:
# app category 넣기
cat = pd.read_csv("data/0226_app_category.csv", encoding = "cp949")
# app_name과 app_name2 에 있어서 두 dictionary 합쳐줌 (안드로이드 - ios 예상?)
cat_dict = cat[["app_name","category"]].set_index("app_name").to_dict()["category"]
cat_dict2 = cat[["app_name2","category"]].set_index("app_name2").to_dict()["category"]
cat_dict.update(cat_dict2)
cat_dict["oil_GS"] = "oil"
cat_dict["oil_SK"] = "oil"
cat_dict["oil_S-oil"] = "oil"
cat_dict["oil_HD"] = "oil"
sub_cat_dict = cat[["app_name","sub_category"]].set_index("app_name").to_dict()["sub_category"]
sub_cat_dict2 = cat[["app_name2","sub_category"]].set_index("app_name2").to_dict()["sub_category"]
sub_cat_dict.update(sub_cat_dict2)
sub_cat_dict["oil_GS"] = "oil"
sub_cat_dict["oil_SK"] = "oil"
sub_cat_dict["oil_S-oil"] = "oil"
sub_cat_dict["oil_HD"] = "oil"
# node 데이터에 붙이기
res_node["cat"] = res_node.apply(lambda x: cat_dict[x["Id"]], axis =1)
res_node["sub_cat"] = res_node.apply(lambda x: sub_cat_dict[x["Id"]], axis =1)
# 카테고리 list 화
cat_list = list(sorted(set(cat_dict.values())))
sub_cat_list = list(sorted(set(sub_cat_dict.values())))
# categorical 숫자 column
res_node["cat_n"] = res_node.apply(lambda x: cat_list.index(x["cat"]), axis =1)
res_node["sub_cat_n"] = res_node.apply(lambda x: sub_cat_list.index(x["sub_cat"]), axis =1)

In [229]:
res.to_csv("data/0226_gephi_0_edge.csv", encoding = "cp949", index= False)
res_node.to_csv("data/0226_gephi_0_node.csv", encoding = "cp949", index= False)

#### 1. 어플 세부카테고리로

In [51]:
app1 = pd.read_csv("data/PANEL/APP Refine_CategoryTime_MME.csv", encoding = "cp949")

In [67]:
# oil 패널들만 남기기
app1["index"] = app1.apply(lambda x: 1 if x["panel_id"] in panel_list else 0 , axis = 1)
app1_oil = app1[app1["index"]==1]
# pivot table
app1_pivot = pd.pivot_table(app1_oil, index = "panel_id", columns = "sub_category", values = "Total_UsageTime", aggfunc=np.mean).reset_index().fillna(0)
# meta data 붙이기
app1_pivot = pd.merge(app1_pivot, meta_data, on ="panel_id")
# 가중치 곱하기
app1_pivot = app1_pivot.drop("panel_id", axis = 1)
app1_w = app1_pivot.apply(lambda x: x * (weight.loc[str(int(x["gender"]))+"-"+str(int(x["age_10"]))]["weight"]), axis = 1)
app1_w = app1_w.drop(["age_10","gender"], axis =1)

In [68]:
# gephi 모양 만들기
cnt = app1_w.copy()

res = pd.DataFrame(columns = [0,1,2,3])
for i in range(cnt.shape[1]-4, cnt.shape[1]):
    for j in range(cnt.shape[1]-4):
        pearr = stats.pearsonr(cnt.iloc[:,i:i+1].iloc[:,0],cnt.iloc[:,j:j+1].iloc[:,0])
        if(pearr[1] < 0.05):
            temp = pd.DataFrame([cnt.columns[i], cnt.columns[j], pearr[0], pearr[1]]).T
            res = res.append(temp)            
res.columns = ["Source","Target","Weight","p-value"]
# size 넣기
oil_list = ["oil_GS","oil_SK","oil_S-oil","oil_HD"]
res_node = pd.DataFrame()
res_node["node"] = list(res["Source"].unique()) + list(res["Target"].unique())
res_node["size"] = res_node.apply(lambda x : 50 if x["node"] in oil_list else 20, axis = 1)
# 하나의 주유소와만 연결된 애들 찾기
temp = res.groupby("Target").count().reset_index()
once_list = list(temp[temp["Source"] == 1]["Target"])
res_node["color"] = res_node.apply(lambda x: 0 if x["node"] in once_list else x["size"], axis = 1)
res_node.columns = ["Id", "size","color"]
res_node["Label"] = res_node["Id"]

In [71]:
res.to_csv("data/0225_gephi_1_edge.csv", encoding = "cp949", index= False)
res_node.to_csv("data/0225_gephi_1_node.csv", encoding = "cp949", index= False)

#### 2. 어플 카테고리로 묶어서

In [21]:
app2 = pd.read_csv("data/PANEL/APP Refine_CategoryTime_MME.csv", encoding = "cp949")

In [22]:
# oil 패널들만 남기기
app2["index"] = app2.apply(lambda x: 1 if x["panel_id"] in panel_list else 0 , axis = 1)
app2_oil = app2[app2["index"]==1]
# pivot table
app2_pivot = pd.pivot_table(app2_oil, index = "panel_id", columns = "category", values = "Total_UsageTime", aggfunc=np.mean).reset_index().fillna(0)
# meta data 붙이기
app2_pivot = pd.merge(app2_pivot, meta_data, on ="panel_id")
# 가중치 곱하기
app2_pivot = app2_pivot.drop("panel_id", axis = 1)
app2_w = app2_pivot.apply(lambda x: x * (weight.loc[str(int(x["gender"]))+"-"+str(int(x["age_10"]))]["weight"]), axis = 1)
app2_w = app2_w.drop(["age_10","gender"], axis =1)

In [31]:
# gephi 모양 만들기
cnt = app2_w.copy()

res = pd.DataFrame(columns = [0,1,2,3])
for i in range(cnt.shape[1]-4, cnt.shape[1]):
    for j in range(cnt.shape[1]-4):
        pearr = stats.pearsonr(cnt.iloc[:,i:i+1].iloc[:,0],cnt.iloc[:,j:j+1].iloc[:,0])
        if(pearr[1] < 0.05):
            temp = pd.DataFrame([cnt.columns[i], cnt.columns[j], pearr[0], pearr[1]]).T
            res = res.append(temp)            
res.columns = ["Source","Target","Weight","p-value"]
# size 넣기
oil_list = ["oil_GS","oil_SK","oil_S-oil","oil_HD"]
res_node = pd.DataFrame()
res_node["node"] = list(res["Source"].unique()) + list(res["Target"].unique())
res_node["size"] = res_node.apply(lambda x : 50 if x["node"] in oil_list else 20, axis = 1)
# 하나의 주유소와만 연결된 애들 찾기
temp = res.groupby("Target").count().reset_index()
once_list = list(temp[temp["Source"] == 1]["Target"])
res_node["color"] = res_node.apply(lambda x: 0 if x["node"] in once_list else x["size"], axis = 1)
res_node.columns = ["Id", "size","color"]
res_node["Label"] = res_node["Id"]

In [33]:
res.to_csv("data/0225_gephi_2_edge.csv", encoding = "cp949", index= False)
res_node.to_csv("data/0225_gephi_2_node.csv", encoding = "cp949", index= False)

#### 3. 결제 카테고리별

In [231]:
pay3 = pd.read_csv("data/PANEL/PAYMENT Summary_Category.csv", encoding = "cp949")

In [147]:
brand_code = pd.read_csv("data/0225_additional_cat.csv", encoding="cp949")

In [148]:
# 카테고리알기위한 dictionary 만들기 (categorycode - categoryname)
temp = brand_code[["code","Category"]]
temp = temp.set_index("code")
category_dict = temp.to_dict()["Category"]
category_dict["oil_GS"] = "oil_GS"
category_dict["oil_SK"] = "oil_SK"
category_dict["oil_S-oil"] = "oil_S-oil"
category_dict["oil_HD"] = "oil_HD"

In [149]:
# oil 패널들만 남기기
pay3["index"] = pay3.apply(lambda x: 1 if x["PANEL_ID"] in panel_list else 0 , axis = 1)
pay3_oil = pay3[pay3["index"]==1]
# pivot table
pay3_pivot = pd.pivot_table(pay3_oil, index = "PANEL_ID", columns = "CATEGORY_CODE", values = "CATEGORY_CNT", aggfunc=np.mean).reset_index().fillna(0)
# meta data 붙이기
meta_data.columns = ['PANEL_ID', 'gender', 'age_10', 'oil_GS', 'oil_SK', 'oil_S-oil','oil_HD']
pay3_pivot = pd.merge(pay3_pivot, meta_data, on ="PANEL_ID")
# 가중치 곱하기
pay3_pivot = pay3_pivot.drop("PANEL_ID", axis = 1)
pay3_w = pay3_pivot.apply(lambda x: x * (weight.loc[str(int(x["gender"]))+"-"+str(int(x["age_10"]))]["weight"]), axis = 1)
pay3_w = pay3_w.drop(["age_10","gender"], axis =1)
pay3_w.columns = [category_dict[x] for x in pay3_w.columns]

In [151]:
# gephi 모양 만들기
cnt = pay3_w.copy()

res = pd.DataFrame(columns = [0,1,2,3])
for i in range(cnt.shape[1]-4, cnt.shape[1]):
    for j in range(cnt.shape[1]-4):
        pearr = stats.pearsonr(cnt.iloc[:,i:i+1].iloc[:,0],cnt.iloc[:,j:j+1].iloc[:,0])
        if(pearr[1] < 0.05):
            temp = pd.DataFrame([cnt.columns[i], cnt.columns[j], pearr[0], pearr[1]]).T
            res = res.append(temp)            
res.columns = ["Source","Target","Weight","p-value"]
# size 넣기
oil_list = ["oil_GS","oil_SK","oil_S-oil","oil_HD"]
res_node = pd.DataFrame()
res_node["node"] = list(res["Source"].unique()) + list(res["Target"].unique())
res_node["size"] = res_node.apply(lambda x : 50 if x["node"] in oil_list else 20, axis = 1)
# 하나의 주유소와만 연결된 애들 찾기
temp = res.groupby("Target").count().reset_index()
once_list = list(temp[temp["Source"] == 1]["Target"])
res_node["color"] = res_node.apply(lambda x: 0 if x["node"] in once_list else x["size"], axis = 1)
res_node.columns = ["Id", "size","color"]
res_node["Label"] = res_node["Id"]

In [152]:
res.to_csv("data/0225_gephi_3_edge.csv", encoding = "cp949", index= False)
res_node.to_csv("data/0225_gephi_3_node.csv", encoding = "cp949", index= False)

In [154]:
res.shape

(61, 4)

#### 4. 결제 store 별

In [54]:
pay4 = pd.read_csv("data/0225_payment_brandname.csv", encoding = "cp949")
del pay4["Unnamed: 0"]

In [55]:
brand_code = pd.read_csv("data/0225_brand_code.csv", encoding="cp949")

In [56]:
# 카테고리알기위한 dictionary 만들기
temp = brand_code[["BrandName","CategoryName"]]
temp = temp.set_index("BrandName")
category_dict = temp.to_dict()["CategoryName"]
category_dict["oil_GS"] = "oil"
category_dict["oil_SK"] = "oil"
category_dict["oil_S-oil"] = "oil"
category_dict["oil_HD"] = "oil"

In [57]:
# oil 패널들만 남기기
pay4["index"] = pay4.apply(lambda x: 1 if x["PANEL_ID"] in panel_list else 0 , axis = 1)
pay4_oil = pay4[pay4["index"]==1]
# pivot table
pay4_pivot = pd.pivot_table(pay4_oil, index = "PANEL_ID", columns = "BrandName", values = "BRAND_CNT", aggfunc=np.mean).reset_index().fillna(0)
# meta data 붙이기
meta_data.columns = ['PANEL_ID', 'gender', 'age_10', 'oil_GS', 'oil_SK', 'oil_S-oil','oil_HD']
pay4_pivot = pd.merge(pay4_pivot, meta_data, on ="PANEL_ID")
# 가중치 곱하기
pay4_pivot = pay4_pivot.drop("PANEL_ID", axis = 1)
pay4_w = pay4_pivot.apply(lambda x: x * (weight.loc[str(int(x["gender"]))+"-"+str(int(x["age_10"]))]["weight"]), axis = 1)
pay4_w = pay4_w.drop(["age_10","gender"], axis =1)

In [86]:
# gephi 모양 만들기
cnt = pay4_w.copy()

res = pd.DataFrame(columns = [0,1,2,3])
for i in range(cnt.shape[1]-4, cnt.shape[1]):
    for j in range(cnt.shape[1]-4):
        pearr = stats.pearsonr(cnt.iloc[:,i:i+1].iloc[:,0],cnt.iloc[:,j:j+1].iloc[:,0])
        if(pearr[1] < 0.05):
            temp = pd.DataFrame([cnt.columns[i], cnt.columns[j], pearr[0], pearr[1]]).T
            res = res.append(temp)            
res.columns = ["Source","Target","Weight","p-value"]
# size 넣기
oil_list = ["oil_GS","oil_SK","oil_S-oil","oil_HD"]
res_node = pd.DataFrame()
res_node["node"] = list(res["Source"].unique()) + list(res["Target"].unique())
res_node["size"] = res_node.apply(lambda x : 50 if x["node"] in oil_list else 20, axis = 1)
# 하나의 주유소와만 연결된 애들 찾기
temp = res.groupby("Target").count().reset_index()
once_list = list(temp[temp["Source"] == 1]["Target"])
res_node["color"] = res_node.apply(lambda x: 0 if x["node"] in once_list else x["size"], axis = 1)
res_node.columns = ["Id", "size","color"]
res_node["Label"] = res_node["Id"]
res_node["category"] = res_node.apply(lambda x: category_dict[x["Id"]], axis = 1)
# 카테고리 숫자로 매칭
temp = res_node.groupby("category").count().reset_index()
temp["Id"]= range(26)
temp = temp[["category","Id"]]
toID_list = temp.set_index("category").to_dict()["Id"]
toID_list["oil"] = 26
res_node["category_n"] = res_node.apply(lambda x: toID_list[x["category"]], axis = 1)

In [73]:
# 16 - 온라인쇼핑만 그리기
res["index"] = res.apply(lambda x: 1 if toID_list[category_dict[x["Target"]]] == 16 else 0, axis =1)
res = res[res["index"]==1]
del res["index"]

res_node["index"] = res_node.apply(lambda x: 1 if (x["category"] == "온라인쇼핑" or x["category"] == "oil") else 0, axis =1)
res_node= res_node[res_node["index"] == 1]
del res_node["index"]

res.to_csv("data/0225_gephi_4_16_edge.csv", encoding = "cp949", index= False)
res_node.to_csv("data/0225_gephi_4_16_node.csv", encoding = "cp949", index= False)

In [87]:
# 6 - 보험 그리기
res["index"] = res.apply(lambda x: 1 if toID_list[category_dict[x["Target"]]] == 6 else 0, axis =1)
res = res[res["index"]==1]
del res["index"]

res_node["index"] = res_node.apply(lambda x: 1 if (x["category"] == "보험료" or x["category"] == "oil") else 0, axis =1)
res_node= res_node[res_node["index"] == 1]
del res_node["index"]

res.to_csv("data/0225_gephi_4_6_edge.csv", encoding = "cp949", index= False)
res_node.to_csv("data/0225_gephi_4_6_node.csv", encoding = "cp949", index= False)

In [85]:
res.to_csv("data/0225_gephi_4_edge.csv", encoding = "cp949", index= False)
res_node.to_csv("data/0225_gephi_4_node.csv", encoding = "cp949", index= False)